In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["HF_TOKKEN"] = os.getenv("HF_TOKKEN")

In [5]:
from langchain_core.documents import Document
from langchain_groq import ChatGroq
llm = ChatGroq(model="Llama3-8b-8192")

In [6]:
documents = [
    Document(page_content="Goldfish are popular pets for begginers, requiring relatively simple care",metadata={"source":"fish-pets-doc"}),
    Document(page_content="Parrots are inteligent birds capable of mimicking human speech",metadata={"source":"bird-pets-doc"}),
    Document(page_content="Rabbits are social animals that need plenty of space to hop around",metadata={"source":"mammal-pets-doc"}),
    Document(page_content="Cats are independent pets that often enjoy their own space.",metadata={"source":"mammal-pets-doc"}),
    Document(page_content="Dogs are great companions, known for their loaylty and friendship",metadata={"source":"mammal-pets-doc"}),
]

In [7]:
documents

[Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for begginers, requiring relatively simple care'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are inteligent birds capable of mimicking human speech'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loaylty and friendship')]

In [8]:
type(documents[0])

langchain_core.documents.base.Document

In [9]:
from langchain_community.vectorstores import Chroma

In [15]:
!pip install langchain_huggingface

ERROR: Could not find a version that satisfies the requirement langchain_huggingface (from versions: none)
ERROR: No matching distribution found for langchain_huggingface


In [10]:
from langchain_openai import OpenAIEmbeddings

In [11]:
embedder = OpenAIEmbeddings(model="text-embedding-3-large")

In [25]:
embedder

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000029600A6E8D0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002967C5D6CD0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [12]:
db = Chroma.from_documents(documents,embedder)

In [13]:
db.similarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loaylty and friendship'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are inteligent birds capable of mimicking human speech'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for begginers, requiring relatively simple care')]

In [16]:
await db.asimilarity_search("dog")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loaylty and friendship'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are inteligent birds capable of mimicking human speech'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around')]

In [17]:
db.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  1.3621606826782227),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loaylty and friendship'),
  1.5692647695541382),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are inteligent birds capable of mimicking human speech'),
  1.6050323247909546),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for begginers, requiring relatively simple care'),
  1.607610821723938)]

In [19]:
retriever = db.as_retriever(search_type="similarity",
                search_kwargs={"k":1})

In [20]:
retriever.batch(["cat","dog"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loaylty and friendship')]]

In [21]:
message = """
Answer the following given question by considering the given context
{questions}
Context:
{context}
"""

In [29]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages({'human',message})

In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
outpt_parser = StrOutputParser()
rag_chain = {"context":retriever,"questions":RunnablePassthrough()}|prompt|llm|outpt_parser

In [37]:
response = rag_chain.invoke("tell me about dogs")

In [38]:
print(response)

Based on the provided context, here's what I can tell you about dogs:

Dogs are amazing companions, renowned for their unwavering loyalty and friendship.
